In [1]:
!pip install -q kaggle


In [2]:
from google.colab import files

# Upload kaggle API key file
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


ModuleNotFoundError: No module named 'google'

In [ ]:
!kaggle competitions download -c tpu-getting-started


In [ ]:
import os
import zipfile

# Unzip the dataset (the path might be different based on the dataset's structure)
with zipfile.ZipFile('tpu-getting-started.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

# Check the dataset directory
os.listdir('data')


In [ ]:
import tensorflow as tf


In [ ]:
# Function to parse TFRecord files
def _parse_function(proto):
    # Define your `features` dictionary here and parse `proto`
    features = {'image/encoded': tf.io.FixedLenFeature([], tf.string),
                'image/class/label': tf.io.FixedLenFeature([], tf.int64)}
    parsed_features = tf.io.parse_single_example(proto, features)
    # Decode the JPEG image
    image = tf.io.decode_jpeg(parsed_features['image/encoded'], channels=3)
    image = tf.image.resize(image, [331, 331])  # Match the size of the images in the TFRecord
    image = image / 255.0  # Normalize the pixel values
    label = tf.cast(parsed_features['image/class/label'], tf.int32)
    label = tf.one_hot(label, depth=104)  # Assuming there are 104 classes
    return image, label

# Function to load dataset
def load_dataset(filepaths):
    dataset = tf.data.TFRecordDataset(filepaths)
    dataset = dataset.map(_parse_function)  # Parse the record into tensors
    dataset = dataset.shuffle(1024).batch(32)  # Shuffle and batch the dataset
    return dataset

# Assuming you want to use the 331x331 resolution
train_files = tf.io.gfile.glob('data/tfrecords-jpeg-331x331/train/*.tfrec')
val_files = tf.io.gfile.glob('data/tfrecords-jpeg-331x331/val/*.tfrec')  # Validation data

train_dataset = load_dataset(train_files)
val_dataset = load_dataset(val_files)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(331, 331, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(104, activation='softmax')  # Update this to 104 to match the number of classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_dataset, epochs=5, validation_data=val_dataset)


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.title('Training accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()


In [ ]:
# Experimenting with a deeper network
deeper_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(5, activation='softmax')
])

deeper_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.optimizers import Adam

# Trying a different optimizer and learning rate
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Experimenting with different data augmentation parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
import json

# Define a simple logger
def log_experiment(experiment_name, history):
    # Save the metrics from the `history` object into a file
    with open(f'{experiment_name}_log.json', 'w') as f:
        json.dump(history.history, f)

# After training an experiment
history = deeper_model.fit(train_generator, epochs=5)
log_experiment('deeper_model', history)


In [ ]:
%load_ext tensorboard
import datetime

# Start a new experiment log
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model with the callback
history = model.fit(
    train_generator,
    epochs=5,
    callbacks=[tensorboard_callback]
)

# You can then view the TensorBoard logs within Colab:
%tensorboard --logdir logs/fit


In [ ]:
# Assuming you have logged several experiments in JSON files
import os

# Plot all the experiments
plt.figure(figsize=(10, 6))
for experiment_log in os.listdir('logs'):
    if experiment_log.endswith('_log.json'):
        with open(f'logs/{experiment_log}', 'r') as f:
            history = json.load(f)
            plt.plot(history['accuracy'], label=experiment_log.replace('_log.json', ''))

plt.title('Comparison of experiment accuracies')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
